In [1]:
experimentsPath= "/raid/elhamod/Butterflies2/experiments/" # "/raid/elhamod/CIFAR_HGNN/experiments/"
dataPath="/raid/elhamod/Butterflies2/Datasets" #"/raid/elhamod/Fish" "/raid/elhamod/"
experimentName = "Butterflies_round1_1" 

projectName_wandb = 'Butterflies'

device = 0
detailed_reporting = False

In [2]:
import matplotlib.pyplot as plt
import torch
import sys
import os
from sklearn.metrics import f1_score
import pandas as pd
from tqdm import tqdm
from tqdm.auto import trange
import wandb
from torchsummary import summary

import warnings
# warnings.filterwarnings("ignore")

try:
    import wandb
except:
    print('wandb not found')

from myhelpers import config_plots, TrialStatistics
from myhelpers.try_warning import try_running
from Butterflies.train import CNN, dataLoader
from myhelpers import cifar_dataLoader
from Butterflies.train.configParser import ConfigParser, getModelName, getDatasetName
config_plots.global_settings()
from myhelpers.seeding import seed, get_seed_from_trialNumber

experimetnsFileName = "experiments.csv"
WANDB_message="wandb not working"


# For logging to server
try_running(lambda : wandb.login(), WANDB_message)

experimentPathAndName = os.path.join(experimentsPath, experimentName)
# set cuda
if device is not None:
    print("using cuda", device)
    torch.cuda.set_device(device)

else:
    print("using cpu")

# get experiment params
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)

# init experiments file
experimentsFileNameAndPath = os.path.join(experimentsPath, experimetnsFileName)

paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)  
experiment_index = 0

# Loop through experiments
# with progressbar.ProgressBar(max_value=number_of_experiments) as bar:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for experiment_params in config_parser.getExperiments():
        print(experiment_params)
        experimentHash =TrialStatistics.getTrialName(experiment_params)

        # load images 
        if experiment_params['image_path'] == 'cifar-100-python':
            datasetManager = cifar_dataLoader.datasetManager(experimentPathAndName, dataPath)
        else:
            datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)
        datasetManager.updateParams(config_parser.fixPaths(experiment_params))

        # Loop through n trials
        for i in trange(experiment_params["numOfTrials"], desc="trial"):
            # We need to set the seed so that whenever exactly the same model params are used, we get same results.
            # SEED_INT = get_seed_from_Hex(trialHash)

            # We want the seed to only be dependent on trial number 
            SEED_INT = get_seed_from_trialNumber(i)
            if detailed_reporting:
                print("used seed: ", SEED_INT)
            seed(SEED_INT)
    
            train_loader, validation_loader, test_loader = datasetManager.getLoaders(SEED_INT)
            architecture = CNN.get_architecture(experiment_params, train_loader.dataset.csv_processor)
            
            modelName = getModelName(experiment_params, i)
            trialName = os.path.join(experimentPathAndName, modelName)
            trialHash = TrialStatistics.getTrialName(experiment_params, i)

            row_information = {
                'experimentName': experimentName,
                'modelName': modelName,
                'datasetName': getDatasetName(config_parser.fixPaths(experiment_params)),
                'experimentHash': experimentHash,
                'trialHash': trialHash
            }
            row_information = {**row_information, **experiment_params} 
            print(row_information)

            run = try_running(lambda : wandb.init(project=projectName_wandb, group=experimentName+"-"+experimentHash, name=trialHash, config=row_information), WANDB_message) #, reinit=True

            # Train/Load model
            model = CNN.create_model(architecture, experiment_params, device=device)

            if detailed_reporting:
                print(model)
                summary(model, (3, 448, 448))
                try_running(lambda : wandb.watch(model, log="all"), WANDB_message)

            if os.path.exists(CNN.getModelFile(trialName)):
                print("Model {0} found!".format(trialName))
            else:
                initModelPath = CNN.getInitModelFile(experimentPathAndName)
                if os.path.exists(initModelPath):
                    model.load_state_dict(torch.load(initModelPath))
                    print("Init Model {0} found!".format(initModelPath))
                CNN.trainModel(train_loader, validation_loader, experiment_params, model, trialName, test_loader, device=device, detailed_reporting=detailed_reporting)

            # Add to experiments file
            if os.path.exists(experimentsFileNameAndPath):
                experiments_df = pd.read_csv(experimentsFileNameAndPath)
            else:
                experiments_df = pd.DataFrame()

            record_exists = not (experiments_df[experiments_df['modelName'] == modelName][experiments_df['experimentName'] == experimentName]).empty if not experiments_df.empty else False
            if record_exists:
                experiments_df.drop(experiments_df[experiments_df['modelName'] == modelName][experiments_df['experimentName'] == experimentName].index, inplace = True) 

            experiments_df = experiments_df.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
            experiments_df.to_csv(experimentsFileNameAndPath, header=True, index=False)

            try_running(lambda : run.finish(), WANDB_message)

        bar.update()

        experiment_index = experiment_index + 1



# if __name__ == "__main__":
#     torch.multiprocessing.set_start_method('spawn')
    
#     import argparse

#     parser = argparse.ArgumentParser()
#     parser.add_argument('--cuda', required=True, type=int)
#     parser.add_argument('--experiments', required=True)
#     parser.add_argument('--data', required=True)
#     parser.add_argument('--name', required=True)
#     parser.add_argument('--detailed', required=False, action='store_true')
#     args = parser.parse_args()
#     main(experimentName=args.name, experimentsPath=args.experiments, dataPath=args.data, device=args.cuda, detailed_reporting=args.detailed)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mndhamod (use `wandb login --relogin` to force relogin)
experiment:   0%|          | 0/5 [00:00<?, ?it/s]

using cuda 0
{'image_path': 'LowResolution', 'suffix': 'preprocessed', 'useCrossValidation': False, 'img_res': 224, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.001, 'numOfTrials': 3, 'fc_layers': 1, 'pretrained': True, 'epochs': 120, 'patience': -1, 'optimizer': 'adabelief', 'scheduler': 'plateau', 'weightdecay': 0.01, 'scheduler_gamma': 0.5, 'scheduler_patience': 15, 'modelType': 'BB', 'lambda': 1, 'two_phase_lambda': False, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': False, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.5, 'tripletEnabled': True, 'regularTripletLoss': False, 'tripletSamples': 8, 'tripletSelector': 'semihard', 'tripletMargin': 2, 'triplet_layers_dic': 'layer2,layer4', 'L1reg': False, 'phylogeny_loss': False, 'grayscale': False, 'tl_extralayer': False, 'random_fitting': False, 'phyloDistances': '0.75,0.5,0.25', 'addKernelOrthogonality': False, 'displayName': 'batch_64', 'noSpeciesBackprop': False, 'phylogeny_loss_epsilon': 0.03}


Creating datasets...
Creating datasets... Done.
Creating loaders...
Creating loaders... Done.
{'experimentName': 'Butterflies_round1_1', 'modelName': 'models/4fe35073fb50bbfff99cac9d8fcde0be936515bafeedcf9b990fb6f7', 'datasetName': 'datasplits/20f0713aa829f297aa689b1adefc4e7fe8a5024c8ffa1ae514fc5a6b', 'experimentHash': '54e3be61927e7e3721d482fe2550b14a01929adbaa1039485b8fbd49', 'trialHash': '4fe35073fb50bbfff99cac9d8fcde0be936515bafeedcf9b990fb6f7', 'image_path': 'LowResolution', 'suffix': 'preprocessed', 'useCrossValidation': False, 'img_res': 224, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.001, 'numOfTrials': 3, 'fc_layers': 1, 'pretrained': True, 'epochs': 120, 'patience': -1, 'optimizer': 'adabelief', 'scheduler': 'plateau', 'weightdecay': 0.01, 'scheduler_gamma': 0.5, 'scheduler_patience': 15, 'modelType': 'BB', 'lambda': 1, 'two_phase_lambda': False, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': False, 'adaptive_lambda': 0.1, 'adaptive_alpha':

wandb: wandb version 0.12.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief
Training started...


...

wandb: wandb version 0.12.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief
Training started...


epoch,119
learning rate,0.0001
test_fine_f1,0.95096
test_fine_acc,0.9472
training_fine_acc,1.0
training_fine_f1,1.0
training_loss,1.96462
validation_loss,2.03142
validation_fine_f1,0.9679
_runtime,1821
_timestamp,1642664589


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
learning rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_fine_acc,▁▁▂▃▄▅▆▆▇▇▇▇▇███████████████████████████
training_fine_f1,▁▁▂▃▄▅▆▆▇▇▇▇▇███████████████████████████
training_loss,███▇▇▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss,███▇▇▆▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_fine_f1,▁▁▂▂▄▅▅▆▆▆▇▇▇▇▇▇▇███████████████████████
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▅▆▅▄▃▃▃▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


experiment: 100%|██████████| 5/5 [7:49:44<00:00, 5636.97s/it]  